## Movie Recommender Project Based off Tf-idf matrix

In [2]:
#importing packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import re
import string
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
# !spacy download en_core_web_lg

In [17]:
# !pip install tweet-preprocessor

In [3]:
df=pd.read_csv('links.csv')

In [4]:
df1=pd.read_csv('movies.csv')

In [5]:
df3=pd.read_csv('tags.csv')

### EDA

In [6]:
df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
df1.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# Building the Tf-Idf Rec System

In [18]:
#groupby and drop column to organize df into movieId
df4 = df3.groupby(['movieId']).apply(lambda x : x.sort_values(by=['movieId'], ascending=True))

In [12]:
df4.drop(['movieId'], axis=1, inplace=True)

In [13]:
#merged df with tags to df with titles
test_df= pd.merge(df4, df1, on='movieId', how='inner')

In [16]:
test_df.head()

,movieId,userId,tag,timestamp,title,genres
0,1,261,animation,1313154708,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,155512,imdb top 250,1264024095,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,1,155512,humorous,1264024094,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,1,155512,funny,1264024092,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,1,155512,family,1264024089,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [28]:
#putting all the tags into one movie entry
#Please run this cell if you plan to use the data. Could not be pushed to Github bc of filesize


# from tqdm import tqdm
# tqdm.pandas() 
# results = []

# for ele in tqdm(test_df.title):
#     results.append(test_df[test_df.title==str(ele)].tag)

100%|██████████| 586994/586994 [3:57:34<00:00, 41.18it/s]  


In [27]:
results

[0         animation
 1      imdb top 250
 2          humorous
 3             funny
 4            family
            ...     
 543        children
 544          comedy
 545         fantasy
 546           Pixar
 547           pixar
 Name: tag, Length: 548, dtype: object,
 0         animation
 1      imdb top 250
 2          humorous
 3             funny
 4            family
            ...     
 543        children
 544          comedy
 545         fantasy
 546           Pixar
 547           pixar
 Name: tag, Length: 548, dtype: object,
 0         animation
 1      imdb top 250
 2          humorous
 3             funny
 4            family
            ...     
 543        children
 544          comedy
 545         fantasy
 546           Pixar
 547           pixar
 Name: tag, Length: 548, dtype: object,
 0         animation
 1      imdb top 250
 2          humorous
 3             funny
 4            family
            ...     
 543        children
 544          comedy
 545         fantas

In [29]:
#saving pickle file of results list
# import pickle
# with open ('results.pickle', 'wb') as f:
#     pickle.dump(results, f, 5)

In [30]:
import pickle
with open('results.pickle', 'rb') as file:
      
    # Call load method to deserialze
    results = pickle.load(file)
  

In [32]:
#turning list into pd.Series
df7=pd.Series(results)
df7[0:5]

0    0         animation
1      imdb top 250
2     ...
1    0         animation
1      imdb top 250
2     ...
2    0         animation
1      imdb top 250
2     ...
3    0         animation
1      imdb top 250
2     ...
4    0         animation
1      imdb top 250
2     ...
dtype: object

In [33]:
#concatenating with main df
frames=[test_df.loc[:],df7]
work=pd.concat(frames, axis=1)
work

,movieId,userId,tag,timestamp,title,genres,0
0,1,261,animation,1313154708,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0 animation 1 imdb top 250 2 ...
1,1,155512,imdb top 250,1264024095,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0 animation 1 imdb top 250 2 ...
2,1,155512,humorous,1264024094,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0 animation 1 imdb top 250 2 ...
3,1,155512,funny,1264024092,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0 animation 1 imdb top 250 2 ...
4,1,155512,family,1264024089,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0 animation 1 imdb top 250 2 ...
...,...,...,...,...,...,...,...
586989,151677,132513,nudity,1453951859,Last Hippie Standing (2002),(no genres listed),586985 dance 586986 drugs ...
586990,151677,132513,spirituality,1453951836,Last Hippie Standing (2002),(no genres listed),586985 dance 586986 drugs ...
586991,151677,132513,trippy,1453951839,Last Hippie Standing (2002),(no genres listed),586985 dance 586986 drugs ...
586992,151677,132513,Yogi,1453951834,Last Hippie Standing (2002),(no genres listed),586985 dance 586986 drugs ...


In [34]:
#cleaning up df
work.drop_duplicates(subset='movieId', inplace=True)

In [35]:
#renaming column to tags
work.rename(columns={0:'tags'}, inplace=True)

# Cleaning tags and genre column

In [36]:
import re
import string

In [37]:
work.genres=[str(x).translate(str.maketrans('', '', string.punctuation)) for x in work.tags[:]]

In [38]:
work

,movieId,userId,tag,timestamp,title,genres,tags
0,1,261,animation,1313154708,Toy Story (1995),0 animation\n1 imdb top 250\n2 ...,0 animation 1 imdb top 250 2 ...
548,2,1040,itaege,1438050732,Jumanji (1995),548 itaege\n549 scary...,548 itaege 549 scary ...
687,3,32298,sequel,1385355832,Grumpier Old Men (1995),687 sequel\n68...,687 sequel 688...
705,4,43484,chick flick,1308558977,Waiting to Exhale (1995),705 chick flick\n706 revenge\n707 ...,705 chick flick 706 revenge 707 ...
712,5,41136,humorous,1438507539,Father of the Bride Part II (1995),712 humorous\n...,712 humorous 7...
...,...,...,...,...,...,...,...
586971,151567,206004,aliens,1453647533,The War in Space (1977),586971 aliens\n586972 hostile al...,586971 aliens 586972 hostile ali...
586976,151599,73352,crowd funding,1453694405,Capital C (2015),586976 crowd funding\n586977 Documenta...,586976 crowd funding 586977 Documentar...
586979,151621,119408,Australia,1453758146,Looking for Grace (2015),586979 Australia\n586980 Couple...,586979 Australia 586980 Couple ...
586985,151677,132513,dance,1453951844,Last Hippie Standing (2002),586985 dance\n586986 drugs...,586985 dance 586986 drugs ...


In [39]:
work

,movieId,userId,tag,timestamp,title,genres,tags
0,1,261,animation,1313154708,Toy Story (1995),0 animation\n1 imdb top 250\n2 ...,0 animation 1 imdb top 250 2 ...
548,2,1040,itaege,1438050732,Jumanji (1995),548 itaege\n549 scary...,548 itaege 549 scary ...
687,3,32298,sequel,1385355832,Grumpier Old Men (1995),687 sequel\n68...,687 sequel 688...
705,4,43484,chick flick,1308558977,Waiting to Exhale (1995),705 chick flick\n706 revenge\n707 ...,705 chick flick 706 revenge 707 ...
712,5,41136,humorous,1438507539,Father of the Bride Part II (1995),712 humorous\n...,712 humorous 7...
...,...,...,...,...,...,...,...
586971,151567,206004,aliens,1453647533,The War in Space (1977),586971 aliens\n586972 hostile al...,586971 aliens 586972 hostile ali...
586976,151599,73352,crowd funding,1453694405,Capital C (2015),586976 crowd funding\n586977 Documenta...,586976 crowd funding 586977 Documentar...
586979,151621,119408,Australia,1453758146,Looking for Grace (2015),586979 Australia\n586980 Couple...,586979 Australia 586980 Couple ...
586985,151677,132513,dance,1453951844,Last Hippie Standing (2002),586985 dance\n586986 drugs...,586985 dance 586986 drugs ...


In [40]:
#using regex to clean the data
work.genres=[re.sub(r"\d+", ' ', x) for x in work.genres[:]]

In [41]:
work.genres=[re.sub(r"\n", '', x) for x in work.genres[:]]

In [42]:
work.genres=[re.sub(r"\s+", ',', x) for x in work.genres[:]]

In [43]:
work.genres=[re.sub(r"\s+", '', x) for x in work.genres[:]]

In [44]:
stuff= work.index

In [45]:
work.genres= [work.genres[x][1:] for x in stuff]

In [46]:
#cleaned tags
work.genres=[x.split(',') for x in work.genres[:]]

In [47]:
# we will pop off the last 4 elements of the list since they are redundant and contain no useful info
trash= [work.genres[x].pop(-1) for x in work.index]

In [48]:
trash= [work.genres[x].pop(-1) for x in work.index]

In [49]:
trash= [work.genres[x].pop(-1) for x in work.index]

In [50]:
trash= [work.genres[x].pop(-1) for x in work.index]

In [177]:
work[['title','genres']]

,title,genres
0,Toy Story (1995),"[animation, imdb, top, humorous, funny, family..."
548,Jumanji (1995),"[itaege, scary, Robin, Williams, fantasy, anim..."
687,Grumpier Old Men (1995),"[sequel, old, people, that, is, actually, funn..."
705,Waiting to Exhale (1995),"[chick, flick, revenge, CLV, characters, girl,..."
712,Father of the Bride Part II (1995),"[humorous, Fantasy, Touching, Comedy, sequel, ..."
...,...,...
586971,The War in Space (1977),"[aliens, hostile, aliens, space, travel, Venus]"
586976,Capital C (2015),"[crowd, funding, Documentary]"
586979,Looking for Grace (2015),"[Australia, Couple, relations, Family, relatio..."
586985,Last Hippie Standing (2002),"[dance, drugs, India, music, nudity, spiritual..."


In [181]:
work.genres[586985]

['dance', 'drugs', 'India', 'music', 'nudity', 'spirituality', 'trippy']

# TF-IDF rec engine

In [78]:
df11= work[['title', 'genres']].reset_index()

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [80]:
df11.drop(columns='index', inplace=True)

In [87]:
s=' '
corpus= [s.join(corpus[i]) for i in range(len(corpus))] 

In [90]:


corpus
# TD-IDF Matrix
X = vectorizer.fit_transform(corpus)

# extracting feature names
tfidf_tokens = vectorizer.get_feature_names()

In [91]:
result = pd.DataFrame(
    data=X.toarray(), 
    index=df11.index, 
    columns=tfidf_tokens
)

result

,aaaa,aaliyah,aamir,aanand,aardman,aardmanname,aaron,aarón,aayesha,aba,...,özer,özgür,østergaard,øªùš,ø³ø,þór,žižek,ызусшфд,惊悚,黃飛鴻china
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21649,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21650,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21651,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(result, result)

In [98]:
cosine_similarities

21654

In [102]:
frame1=pd.Series(df11.title) 
frame2=pd.Series(pd.Series(x) for x in cosine_similarities)
frames=[frame1, frame2]
df12= pd.concat(frames, axis=1)

In [103]:
df12

,title,0
0,Toy Story (1995),0 1.000000 1 0.055651 2 0...
1,Jumanji (1995),0 0.055651 1 1.000000 2 0...
2,Grumpier Old Men (1995),0 0.028662 1 0.000000 2 1...
3,Waiting to Exhale (1995),0 0.000000 1 0.000000 2 0...
4,Father of the Bride Part II (1995),0 0.114811 1 0.016511 2 0...
...,...,...
21649,The War in Space (1977),0 0.0 1 0.0 2 0.0 3 ...
21650,Capital C (2015),0 0.0 1 0.0 2 0.0 3 ...
21651,Looking for Grace (2015),0 0.050315 1 0.000000 2 0...
21652,Last Hippie Standing (2002),0 0.0 1 0.0 2 0.0 3 ...


In [117]:
indices = pd.Series(df11.title)

def recommendations(name, cosine_similarities = cosine_similarities):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the name
    idx = indices[indices == name].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies except itself
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the names of the top 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    Dataframe = [df12[df12.index==x].title for x in recommended_movies]
    return Dataframe

In [119]:
recommendations('3:10 to Yuma (2007)')


[2418    Mystery, Alaska (1999)
 Name: title, dtype: object,
 3392    Proof of Life (2000)
 Name: title, dtype: object,
 267    Quick and the Dead, The (1995)
 Name: title, dtype: object,
 1270    Rough Magic (1995)
 Name: title, dtype: object,
 13106    Fastest Gun Alive, The (1956)
 Name: title, dtype: object,
 13636    New York Confidential (1955)
 Name: title, dtype: object,
 883    For the Moment (1994)
 Name: title, dtype: object,
 8154    Lost Battalion, The (2001)
 Name: title, dtype: object,
 1475    Heaven's Burning (1997)
 Name: title, dtype: object,
 297    Virtuosity (1995)
 Name: title, dtype: object]